In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
df = pd.read_csv("survey_results_public.csv", index_col="Respondent", usecols=["Respondent","MainBranch", "ResumeUpdate"]).dropna()

df["MainBranch"] = df["MainBranch"].str.replace("I am not primarily a developer, but I write code sometimes as part of my work","I am not primarily a developer, but I \nwrite code sometimes as part of my work")
df["MainBranch"] = df["MainBranch"].str.replace('I used to be a developer by profession, but no longer am','I used to be a developer by profession, \nbut no longer am')

df["ResumeUpdate"] = df["ResumeUpdate"].replace("My job status changed (promotion, new job, etc.)","My job status changed \n(promotion, new job, etc.)")
df["ResumeUpdate"] = df["ResumeUpdate"].replace("Something else changed (education, award, media, etc.)","Something else changed \n(education, award, media, etc.)")
df["ResumeUpdate"] = df["ResumeUpdate"].replace("I heard about a job opportunity (from a recruiter, online job posting, etc.)","I heard about a job opportunity \n(from a recruiter, online job posting, etc.)")
df["ResumeUpdate"] = df["ResumeUpdate"].str.replace("I had a negative experience or interaction at work","I had a negative experience or \ninteraction at work")

df["MainBranch"] = df["MainBranch"].astype("category")
df["ResumeUpdate"] = df["ResumeUpdate"].astype("category")

df.head()

MainBranch  \
Respondent                                                      
2                      I am a student who is learning to code   
3           I am not primarily a developer, but I \nwrite ...   
4                              I am a developer by profession   
5                              I am a developer by profession   
6           I am not primarily a developer, but I \nwrite ...   

                                                 ResumeUpdate  
Respondent                                                     
2           Something else changed \n(education, award, me...  
3                            I was preparing for a job search  
4                            I was preparing for a job search  
5                            I was preparing for a job search  
6           I heard about a job opportunity \n(from a recr...

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77877 entries, 2 to 88883
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   MainBranch    77877 non-null  category
 1   ResumeUpdate  77877 non-null  category
dtypes: category(2)
memory usage: 760.8 KB


In [4]:
df_1 = df.groupby(["ResumeUpdate", "MainBranch"]).size().to_frame(name='count').reset_index()
df_1.head()

ResumeUpdate  \
0  I had a negative experience or \ninteraction a...   
1  I had a negative experience or \ninteraction a...   
2  I had a negative experience or \ninteraction a...   
3  I heard about a job opportunity \n(from a recr...   
4  I heard about a job opportunity \n(from a recr...   

                                          MainBranch  count  
0                     I am a developer by profession   5578  
1             I am a student who is learning to code    137  
2  I am not primarily a developer, but I \nwrite ...    557  
3                     I am a developer by profession   9238  
4             I am a student who is learning to code    926

In [5]:
df_1_1 = df_1.groupby("ResumeUpdate")["count"].sum().sort_values(ascending=False).to_frame().reset_index()
df_1_1["percent"] = round((100.*df_1_1["count"]/df_1_1["count"].sum()),1)
df_1_1

ResumeUpdate  count  percent
0                   I was preparing for a job search  33309     42.8
1  My job status changed \n(promotion, new job, e...  13546     17.4
2  Something else changed \n(education, award, me...  11986     15.4
3  I heard about a job opportunity \n(from a recr...  11327     14.5
4  I had a negative experience or \ninteraction a...   6272      8.1
5                        Re-entry into the workforce   1437      1.8

In [6]:
df_1_2 = df_1.groupby("MainBranch")["count"].sum().sort_values(ascending=False).to_frame().reset_index()
df_1_2["percent"] = round((100.*df_1_2["count"]/df_1_2["count"].sum()),1)
df_1_2

MainBranch  count  percent
0                     I am a developer by profession  63660     81.7
1  I am not primarily a developer, but I \nwrite ...   7164      9.2
2             I am a student who is learning to code   7053      9.1

In [7]:
df_2 = df_1.pivot_table(index="MainBranch", columns="ResumeUpdate", values="count")
df_2 = df_2.reindex(df_1_1["ResumeUpdate"], axis=1)  
df_2

ResumeUpdate                                        I was preparing for a job search  \
MainBranch                                                                             
I am a developer by profession                                                 27720   
I am a student who is learning to code                                          3050   
I am not primarily a developer, but I \nwrite c...                              2539   

ResumeUpdate                                        My job status changed \n(promotion, new job, etc.)  \
MainBranch                                                                                               
I am a developer by profession                                                                  11597    
I am a student who is learning to code                                                            606    
I am not primarily a developer, but I \nwrite c...                                               1343    

ResumeUpdate                                        Something else changed \n(education, award, media, etc.)  \
MainBranch                                                                                                     
I am a developer by profession                                                                   8410          
I am a student who is learning to code                                                           2160          
I am not primarily a developer, but I \nwrite c...                                               1416          

ResumeUpdate                                        I heard about a job opportunity \n(from a recruiter, online job posting, etc.)  \
MainBranch                                                                                                                           
I am a developer by profession                                                                   9238                                
I am a student who is learning to code                                                            926                                
I am not primarily a developer, but I \nwrite c...                                               1163                                

ResumeUpdate                                        I had a negative experience or \ninteraction at work  \
MainBranch                                                                                                 
I am a developer by profession                                                                   5578      
I am a student who is learning to code                                                            137      
I am not primarily a developer, but I \nwrite c...                                                557      

ResumeUpdate                                        Re-entry into the workforce  
MainBranch                                                                       
I am a developer by profession                                             1117  
I am a student who is learning to code                                      174  
I am not primarily a developer, but I \nwrite c...                          146

In [8]:
df_2_per = round(((df_2 / df_2.sum().sum())*100),2)
df_2_1 = df_2.astype(str) + '\n' + '(' + df_2_per.astype(str) + ' %)'
df_2_1

ResumeUpdate                                       I was preparing for a job search  \
MainBranch                                                                            
I am a developer by profession                                     27720\n(35.59 %)   
I am a student who is learning to code                               3050\n(3.92 %)   
I am not primarily a developer, but I \nwrite c...                   2539\n(3.26 %)   

ResumeUpdate                                       My job status changed \n(promotion, new job, etc.)  \
MainBranch                                                                                              
I am a developer by profession                                                       11597\n(14.89 %)   
I am a student who is learning to code                                                  606\n(0.78 %)   
I am not primarily a developer, but I \nwrite c...                                     1343\n(1.72 %)   

ResumeUpdate                                       Something else changed \n(education, award, media, etc.)  \
MainBranch                                                                                                    
I am a developer by profession                                                         8410\n(10.8 %)         
I am a student who is learning to code                                                 2160\n(2.77 %)         
I am not primarily a developer, but I \nwrite c...                                     1416\n(1.82 %)         

ResumeUpdate                                       I heard about a job opportunity \n(from a recruiter, online job posting, etc.)  \
MainBranch                                                                                                                          
I am a developer by profession                                                        9238\n(11.86 %)                               
I am a student who is learning to code                                                  926\n(1.19 %)                               
I am not primarily a developer, but I \nwrite c...                                     1163\n(1.49 %)                               

ResumeUpdate                                       I had a negative experience or \ninteraction at work  \
MainBranch                                                                                                
I am a developer by profession                                                         5578\n(7.16 %)     
I am a student who is learning to code                                                  137\n(0.18 %)     
I am not primarily a developer, but I \nwrite c...                                      557\n(0.72 %)     

ResumeUpdate                                       Re-entry into the workforce  
MainBranch                                                                      
I am a developer by profession                                  1117\n(1.43 %)  
I am a student who is learning to code                           174\n(0.22 %)  
I am not primarily a developer, but I \nwrite c...               146\n(0.19 %)

In [9]:
df_3 = df_1.pivot_table(index="ResumeUpdate", columns="MainBranch", values="count")
df_3 = df_3.reindex(df_1_1["ResumeUpdate"], axis=0)
df_3

MainBranch                                          I am a developer by profession  \
ResumeUpdate                                                                         
I was preparing for a job search                                             27720   
My job status changed \n(promotion, new job, etc.)                           11597   
Something else changed \n(education, award, med...                            8410   
I heard about a job opportunity \n(from a recru...                            9238   
I had a negative experience or \ninteraction at...                            5578   
Re-entry into the workforce                                                   1117   

MainBranch                                          I am a student who is learning to code  \
ResumeUpdate                                                                                 
I was preparing for a job search                                                      3050   
My job status changed \n(promotion, new job, etc.)                                     606   
Something else changed \n(education, award, med...                                    2160   
I heard about a job opportunity \n(from a recru...                                     926   
I had a negative experience or \ninteraction at...                                     137   
Re-entry into the workforce                                                            174   

MainBranch                                          I am not primarily a developer, but I \nwrite code sometimes as part of my work  
ResumeUpdate                                                                                                                         
I was preparing for a job search                                                                 2539                                
My job status changed \n(promotion, new job, etc.)                                               1343                                
Something else changed \n(education, award, med...                                               1416                                
I heard about a job opportunity \n(from a recru...                                               1163                                
I had a negative experience or \ninteraction at...                                                557                                
Re-entry into the workforce                                                                       146

In [10]:
df_3_per = round(((df_3 / df_3.sum().sum())*100),2)
df_3_1 = df_3.astype(str) + '\n' + '(' + df_3_per.astype(str) + ' %)'
df_3_1

# df_3_1 = df_3_1.transpose()
# df_3_1

MainBranch                                         I am a developer by profession  \
ResumeUpdate                                                                        
I was preparing for a job search                                 27720\n(35.59 %)   
My job status changed \n(promotion, new job, etc.)               11597\n(14.89 %)   
Something else changed \n(education, award, med...                 8410\n(10.8 %)   
I heard about a job opportunity \n(from a recru...                9238\n(11.86 %)   
I had a negative experience or \ninteraction at...                 5578\n(7.16 %)   
Re-entry into the workforce                                        1117\n(1.43 %)   

MainBranch                                         I am a student who is learning to code  \
ResumeUpdate                                                                                
I was preparing for a job search                                           3050\n(3.92 %)   
My job status changed \n(promotion, new job, etc.)                          606\n(0.78 %)   
Something else changed \n(education, award, med...                         2160\n(2.77 %)   
I heard about a job opportunity \n(from a recru...                          926\n(1.19 %)   
I had a negative experience or \ninteraction at...                          137\n(0.18 %)   
Re-entry into the workforce                                                 174\n(0.22 %)   

MainBranch                                         I am not primarily a developer, but I \nwrite code sometimes as part of my work  
ResumeUpdate                                                                                                                        
I was preparing for a job search                                                       2539\n(3.26 %)                               
My job status changed \n(promotion, new job, etc.)                                     1343\n(1.72 %)                               
Something else changed \n(education, award, med...                                     1416\n(1.82 %)                               
I heard about a job opportunity \n(from a recru...                                     1163\n(1.49 %)                               
I had a negative experience or \ninteraction at...                                      557\n(0.72 %)                               
Re-entry into the workforce                                                             146\n(0.19 %)

<h1 style='color:red'>FUNCTIONS</h1>

In [11]:
######################
# ax1 (top)
######################
def total_bar_graph_annotate(df):
    for idx, acc in enumerate(list(df.columns)): 
        value=df[acc].sum()
        percent = f"({value} - {value*100/df_1['count'].sum():1.1f}%)" 
#         percent = f"({value} - {value*100/df.sum().sum():1.1f}%)" 
        ax1.annotate(percent, xy=(value+300, idx), xycoords="data",   
                        va='center', ha='left', rotation=0)
    
######################
# ax2 (bottom-left)
######################
def table_customization (table, df, index, columns, min, max):
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    cellDict = table.get_celld()

    for i in range(0,len(index)):
        for j in range(0,len(columns)):
            if i==0:
                cellDict[(i,j)].set_height(0.07)
                cellDict[(i,j)].set_width(0.10)

    for i in range(1,len(index)+1):
        cellDict[(i,-1)].set_height(0.17)
        cellDict[(i,-1)].set_width(0.30)
        cellDict[(i,-1)].set_fontsize(10)
        cellDict[(i,-1)].set_text_props(weight='bold', color='k')
        for j in range(0,len(columns)):
            cellDict[(i,j)].set_height(0.17)
            if (i%2 == 0):
                cellDict[(i,j)].set_facecolor('gainsboro')
            else:
                cellDict[(i,j)].set_facecolor('w')

        for i,[j,k] in enumerate(df.iterrows(), start=1):
            for m, l in enumerate(k):
                if (l<min):
                    cellDict[(i,m)].set_facecolor('y')
                elif l>max:
                    cellDict[(i,m)].set_facecolor('g')

######################
# ax3 (bottom-middle)
######################
def autopct(pct, allvals):
    if pct<5:
        return ""
    else:
        absolute = int(pct/100.*sum(allvals))
        return "{p:.1f}%".format(p=pct)
#         return "{p:.1f}% \n({val})".format(p=pct, val=absolute)              
                
def pie_annotate(ax, df, wedges, angleA, x1, y1):
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"), bbox=bbox_props, zorder=0, va="center", ha="center")

    df_filt = df[(df["count"] != 0) & (df["percent"]<5)]
    values = [f"{i:1.1f}%" for i,j in zip(df["percent"], df["count"])]

    for i, p in enumerate(wedges):
        if i in df_filt.index:
            ang = (p.theta2 - p.theta1)/2. + p.theta1
            y = np.sin(np.deg2rad(ang))
            x = np.cos(np.deg2rad(ang))
            horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
            connectionstyle = f"angle,angleA={angleA},angleB={ang}"
            kw["arrowprops"].update({"connectionstyle": connectionstyle})
            ax.annotate(values[i], xy=(x, y), xytext=(x1*np.sign(x), y1*y),
                        horizontalalignment=horizontalalignment, **kw)

<h1 style='color:red'>GRAPHS-1</h1>

In [20]:
%matplotlib inline
fig = plt.figure(constrained_layout=True, figsize=(15,8), facecolor='lightsteelblue')
plt.figtext(0.99, 0.02, "20.09.2020", horizontalalignment='right', size=8, weight='light')
matplotlib.style.use('classic')

widths = [1, 1, 1.2]
heights = [2, 1]
gs_kw = dict(height_ratios=heights)
gs = fig.add_gridspec(ncols=3, nrows=2, width_ratios=widths, height_ratios=heights)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 2])
######################
# ax1 (top)
######################
index = np.arange(len(df_2.columns))
bar_width = 0.5
y_offset = np.zeros(len(df_2.columns))

cmap_red = plt.get_cmap('Reds')
colors_row = cmap_red(np.linspace(0.7, 0.1, len(df_2.columns)))
cmap = plt.get_cmap("Set2")
colors_column = []
for i in range(0,len(df_3.columns)):
    colors_column.append(cmap(i))

cell_text = []
for i in range(len(df_2)):
    ax1.barh(index, df_2.values[i], bar_width, left=y_offset, color=cmap(i), zorder=3)
    y_offset = y_offset + df_2.values[i]
    cell_text.append(['%1.1f' % (x / 1000.0) for x in y_offset])

total_bar_graph_annotate(df_2) #------function
######################
# ax2 (bottom-left)
######################
df_3_empty_col = [""]*(len(df_3.columns)+1)
table_1 = ax2.table(cellText=df_3_1.values, rowLabels=df_3.index, colLabels=df_3_empty_col, loc="upper left", zorder=3, 
                    rowColours=colors_row, colColours=colors_column,)

table_customization(table=table_1, df=df_3, index=df_3.index, columns=df_3.columns, min=100, max=10000) #------function
######################
# ax3 (bottom-middle)
######################
textprops = dict(horizontalalignment="center", verticalalignment="center", rotation=0, rotation_mode="anchor",
                size=12, color="black")
wedgeprops=dict(width=1, edgecolor="w", linewidth=1)

wedges, texts, autotexts = ax3.pie(df_1_1["count"], radius=1, startangle=0, 
                            autopct=lambda pct: autopct(pct, df_1_1["count"]), #------function
                            textprops=textprops, wedgeprops=wedgeprops,
                            counterclock=True, pctdistance = 0.7, colors=colors_row)

pie_annotate(ax3, df_1_1, wedges, 120, 1.5, 1.5) #------function

# Circle
bbox_props = dict(boxstyle="circle,pad=0.3", fc="cyan", ec="b", lw=2)
ax3.text(0, 0, df_3.sum().sum(), ha="center", va="center", rotation=-30, size=13, bbox=bbox_props)  
######################
# ax4 (bottom-right)
######################
labels = [f"({k} - {j:1.1f}%) {i}" for i,j,k in zip(df_1_1["ResumeUpdate"], df_1_1["percent"], df_1_1["count"])]
ax4.legend(wedges, labels, fontsize=10, loc="upper left", prop={'size': 10})
#---------------------------------------------------------------#
########################################################################################
ax1.legend(df_2.index, fontsize=10, bbox_to_anchor=(1,1,0,0))
ax1.set_title("Think back to the last time you updated your resumé, CV, or an online profile on \na job site. What is the PRIMARY reason that you did so?", fontsize=16, color='b', weight='bold')
ax1.grid(axis='x', zorder=0)
ax1.set_yticks(range(len(df_2.columns)))
ax1.set_yticklabels(df_2.columns, fontsize=12)
ax1.set_xlim([0, int(df_1_1["count"].max())*1.20])
# ax1.invert_yaxis()
ax2.axis("off")
ax3.axis("off")
ax3.axis('equal')
ax4.axis("off")
########################################################################################
plt.show()

<h1 style='color:red'>GRAPHS-2</h1>

In [22]:
%matplotlib inline
fig = plt.figure(constrained_layout=True, figsize=(17,8), facecolor='lightsteelblue')
plt.figtext(0.99, 0.02, "20.09.2020", horizontalalignment='right', size=8, weight='light')
matplotlib.style.use('classic')

widths = [1.6, 1, 1.4]
heights = [2, 1]
gs_kw = dict(height_ratios=heights)
gs = fig.add_gridspec(ncols=3, nrows=2, width_ratios=widths, height_ratios=heights)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 2])
######################
# ax1 (top)
######################
index = np.arange(len(df_3.columns))
bar_width = 0.5
y_offset = np.zeros(len(df_3.columns))

cmap_red = plt.get_cmap('Reds')
colors_row = cmap_red(np.linspace(0.7, 0.1, len(df_3.columns)))
cmap = plt.get_cmap("Set2")    
colors_column = []
for i in range(0,len(df_2.columns)):
    colors_column.append(cmap(i))

cell_text = []
for i in range(len(df_3)):
    ax1.barh(index, df_3.values[i], bar_width, left=y_offset, color=cmap(i), zorder=3)
    y_offset = y_offset + df_3.values[i]
    cell_text.append(['%1.1f' % (x / 1000.0) for x in y_offset])

total_bar_graph_annotate(df_3)
######################
# ax2 (bottom-left)
######################
df_2_empty_col = [""]*(len(df_2.columns)+1)
table_2 = ax2.table(cellText=df_2_1.values, rowLabels=df_2.index, colLabels=df_2_empty_col, loc="upper left", zorder=3, 
                    rowColours=colors_row, colColours=colors_column,)

table_customization(table=table_2, df=df_2, index=df_2.index, columns=df_2.columns, min=100, max=10000)
######################
# ax3 (bottom-middle)
######################
textprops = dict(horizontalalignment="center", verticalalignment="center", rotation=0, rotation_mode="anchor",
                size=12, color="black")
wedgeprops=dict(width=1, edgecolor="w", linewidth=1)

wedges, texts, autotexts = ax3.pie(df_1_2["count"], radius=1, startangle=0, 
                            autopct=lambda pct: autopct(pct, df_1_2["count"]), #------function
                            textprops=textprops, wedgeprops=wedgeprops,
                            counterclock=True, pctdistance = 0.7, colors=colors_row)

pie_annotate(ax3, df_1_2, wedges, 135, 1.5, 2.5) #------function

# Circle
bbox_props = dict(boxstyle="circle,pad=0.3", fc="cyan", ec="b", lw=2)
ax3.text(0, 0, df_3.sum().sum(), ha="center", va="center", rotation=-30, size=13, bbox=bbox_props)    

######################
# ax4 (bottom-right)
######################

# labels_OS_1 = [f"({i:}) {j}" for i,j in zip(df_1_2.values, df_1_2.index)]
# ax4.legend(pie[0], labels_OS_1, fontsize=10, loc="upper left")

labels = [f"({k} - {j:1.1f}%) {i}" for i,j,k in zip(df_1_2["MainBranch"], df_1_2["percent"], df_1_2["count"])]
ax4.legend(wedges, labels, fontsize=10, loc="upper left", prop={'size': 10})
#---------------------------------------------------------------#
########################################################################################
ax1.legend(df_3.index, fontsize=10, bbox_to_anchor=(1,1,0,0))
ax1.set_title("Think back to the last time you updated your resumé, CV, or an online profile on \na job site. What is the PRIMARY reason that you did so?", fontsize=16, color='b', weight='bold')
ax1.grid(axis='x', zorder=0)
ax1.set_yticks(range(len(df_3.columns)))
ax1.set_yticklabels(df_3.columns, fontsize=12)
ax1.set_xlim([0, int(df_1_2["count"].max())*1.15])
ax2.axis("off")
ax3.axis('equal')
ax4.axis("off")
########################################################################################
plt.show()